# Notebook Overview: CNN-DINOv2 Hybrid

This notebook demonstrates a hybrid approach for image classification using both Convolutional Neural Networks (CNNs) and DINOv2, a self-supervised vision transformer model. The workflow includes:

- **Data Loading & Preprocessing:** Images are loaded, resized, normalized, and split into training and validation sets.
- **Feature Extraction:** DINOv2 is used to extract high-level features from images, leveraging its transformer-based architecture for robust representations.
- **CNN Model Construction:** A custom CNN is built to process image data, learning spatial hierarchies and patterns.
- **Hybrid Model Integration:** Features from DINOv2 and the CNN are combined, either by concatenation or other fusion techniques, to enhance classification performance.
- **Training & Evaluation:** The hybrid model is trained on the dataset, with metrics such as accuracy and loss tracked. Validation is performed to assess generalization.
- **Visualization & Analysis:** Results, including confusion matrices and sample predictions, are visualized to interpret model behavior.

This approach aims to leverage the strengths of both CNNs (local feature learning) and DINOv2 (global, context-aware representations) for improved image classification results.

# 释放GPU辅助函数

In [ ]:
import torch
import gc

def clear():
    print("\n" + "="*50)
    print("开始释放GPU资源...")
    print("="*50)
    
    # 5. 清空Python垃圾回收
    gc.collect()
    print("✓ Python garbage collected")
    
    # 6. 清空PyTorch缓存
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✓ CUDA cache cleared")
        
        # 7. 同步CUDA设备
        torch.cuda.synchronize()
        print("✓ CUDA synchronized")
        
        # 8. 重置CUDA峰值内存统计
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()
        print("✓ CUDA memory stats reset")
        
        # 9. 显示当前GPU内存使用情况
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            print(f"  GPU {i}: Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB")
    else:
        print("⚠ CUDA not available, skipping CUDA-specific cleanup")
    
    print("="*50)
    print("GPU资源释放完成!")
    print("="*50)

#  第一个submission(0.325)

In [ ]:
import os, cv2, json, math, random, torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from transformers import AutoImageProcessor, AutoModel

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # This forces CUDA to use deterministic algorithms (slower but consistent)
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_DIR  = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
AUTH_DIR  = f"{BASE_DIR}/train_images/authentic"
FORG_DIR  = f"{BASE_DIR}/train_images/forged"
MASK_DIR  = f"{BASE_DIR}/train_masks"
TEST_DIR  = f"{BASE_DIR}/test_images"
DINO_PATH = "/kaggle/input/dinov2/pytorch/base/1"

IMG_SIZE = 518
BATCH_SIZE = 2
# MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U54/CNNDINOv2-U54/model_seg_final.pt'  # 0.310
# MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U52/CNNDINOv2-U52/model_seg_final.pt'  # 0.321
MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U52/CNNDINOv2-U52/model_seg_final.pt'  # 0.321

# INFERENCE UTILS
AREA_THR = 200
MEAN_THR = 0.22
USE_TTA = False
GRID_SEARCH = False

class ForgerySegDataset(Dataset):
    def __init__(self, auth_paths, forg_paths, mask_dir, img_size=IMG_SIZE):
        self.samples = []
        for p in forg_paths:
            m = os.path.join(mask_dir, Path(p).stem + ".npy")
            if os.path.exists(m):
                self.samples.append((p, m))
        for p in auth_paths:
            self.samples.append((p, None))
        self.img_size = img_size
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        w, h = img.size
        if mask_path is None:
            mask = np.zeros((h, w), np.uint8)
        else:
            m = np.load(mask_path)
            if m.ndim == 3: m = np.max(m, axis=0)
            mask = (m > 0).astype(np.uint8)
        img_r = img.resize((IMG_SIZE, IMG_SIZE))
        mask_r = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img_t = torch.from_numpy(np.array(img_r, np.float32)/255.).permute(2,0,1)
        mask_t = torch.from_numpy(mask_r[None, ...].astype(np.float32))
        return img_t, mask_t


#  MODEL (DINOv2 + Decoder)

from transformers import AutoImageProcessor, AutoModel
processor = AutoImageProcessor.from_pretrained(DINO_PATH, local_files_only=True, use_fast=False)
encoder = AutoModel.from_pretrained(DINO_PATH, local_files_only=True).eval().to(device)

class DinoTinyDecoder(nn.Module):
    def __init__(self, in_ch=768, out_ch=1):
        super().__init__()
        # Block 1: 768 -> 384
        self.block1 = nn.Sequential(
            nn.Conv2d(in_ch, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        # Block 2: 384 -> 192
        self.block2 = nn.Sequential(
            nn.Conv2d(384, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        # Block 3: 192 -> 96
        self.block3 = nn.Sequential(
            nn.Conv2d(192, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        # Final Output: 96 -> 1
        self.conv_out = nn.Conv2d(96, out_ch, kernel_size=1)
    
    def forward(self, f, target_size):
        # f: [B, 768, 37, 37]
        
        # Step 1: Up to ~74x74
        x = F.interpolate(self.block1(f), size=(74, 74), mode='bilinear', align_corners=False)
        
        # Step 2: Up to ~148x148
        x = F.interpolate(self.block2(x), size=(148, 148), mode='bilinear', align_corners=False)
        
        # Step 3: Up to ~296x296
        x = F.interpolate(self.block3(x), size=(296, 296), mode='bilinear', align_corners=False)
        
        # Step 4: Final jump to 518x518
        x = self.conv_out(x)
        x = F.interpolate(x, size=target_size, mode='bilinear', align_corners=False)
        
        return x
    
class DinoSegmenter(nn.Module):
    def __init__(self, encoder, processor):
        super().__init__()
        self.encoder, self.processor = encoder, processor
        for p in self.encoder.parameters(): p.requires_grad = False
        self.seg_head = DinoTinyDecoder(768,1)
    def forward_features(self,x):
        imgs = (x*255).clamp(0,255).byte().permute(0,2,3,1).cpu().numpy()
        inputs = self.processor(images=list(imgs), return_tensors="pt").to(x.device)
        # with torch.no_grad(): 
        #     feats = self.encoder(**inputs).last_hidden_state
        feats = self.encoder(**inputs).last_hidden_state
        B,N,C = feats.shape
        fmap = feats[:,1:,:].permute(0,2,1)
        s = int(math.sqrt(N-1))
        fmap = fmap.reshape(B,C,s,s)
        return fmap
    def forward_seg(self,x):
        fmap = self.forward_features(x)
        return self.seg_head(fmap,(IMG_SIZE,IMG_SIZE))


auth_imgs = sorted([str(Path(AUTH_DIR)/f) for f in os.listdir(AUTH_DIR)])
forg_imgs = sorted([str(Path(FORG_DIR)/f) for f in os.listdir(FORG_DIR)])
train_auth, val_auth = train_test_split(auth_imgs, test_size=0.2, random_state=42)
train_forg, val_forg = train_test_split(forg_imgs, test_size=0.2, random_state=42)

train_loader = DataLoader(ForgerySegDataset(train_auth, train_forg, MASK_DIR),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(ForgerySegDataset(val_auth, val_forg, MASK_DIR),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_seg = DinoSegmenter(encoder, processor).to(device)

# Load pretrained weights if MODEL_LOC is specified
if MODEL_LOC is not None and os.path.exists(MODEL_LOC):
    model_seg.load_state_dict(torch.load(MODEL_LOC, map_location=device))
    print(f"✅ Loaded pretrained model from: {MODEL_LOC}")
    model_seg.eval()  # Set model to evaluation mode

@torch.no_grad()
def segment_prob_map(pil):
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    prob = torch.sigmoid(model_seg.forward_seg(x))[0,0].cpu().numpy()
    return prob

@torch.no_grad()
def segment_prob_map_with_tta(pil):
    # 1. Preprocessing: Resize, Normalize, and move to Device
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    
    predictions = []

    # 2. Original Prediction
    pred_orig = torch.sigmoid(model_seg.forward_seg(x))
    predictions.append(pred_orig)

    # 3. Horizontal Flip TTA (dim 3)
    # Flip input -> Predict -> Flip output back
    pred_h = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[3])))
    predictions.append(torch.flip(pred_h, dims=[3]))

    # 4. Vertical Flip TTA (dim 2)
    # Flip input -> Predict -> Flip output back
    pred_v = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[2])))
    predictions.append(torch.flip(pred_v, dims=[2]))

    # 5. Average the predictions and format as numpy
    # We stack the 3 predictions and take the mean across the stack dimension (0)
    prob = torch.stack(predictions).mean(0)[0, 0].cpu().numpy()

    return prob
    
def enhanced_adaptive_mask(prob, alpha_grad=0.45):
    gx = cv2.Sobel(prob, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(prob, cv2.CV_32F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)
    grad_norm = grad_mag / (grad_mag.max() + 1e-6)
    enhanced = (1 - alpha_grad) * prob + alpha_grad * grad_norm
    enhanced = cv2.GaussianBlur(enhanced, (3,3), 0)
    thr = np.mean(enhanced) + 0.3 * np.std(enhanced)
    mask = (enhanced > thr).astype(np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask, thr

def finalize_mask(prob, orig_size):
    mask, thr = enhanced_adaptive_mask(prob)
    mask = cv2.resize(mask, orig_size, interpolation=cv2.INTER_NEAREST)
    return mask, thr

def pipeline_final(pil):
    if USE_TTA:
        prob = segment_prob_map_with_tta(pil)
    else:
        prob = segment_prob_map(pil)
    mask, thr = finalize_mask(prob, pil.size)
    area = int(mask.sum())
    mean_inside = float(prob[cv2.resize(mask,(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_NEAREST)==1].mean()) if area>0 else 0.0
    if area < AREA_THR or mean_inside < MEAN_THR:
        return "authentic", None, {"area": area, "mean_inside": mean_inside, "thr": thr}
    return "forged", mask, {"area": area, "mean_inside": mean_inside, "thr": thr}

import itertools
from sklearn.metrics import f1_score

def grid_search_area_mean(forg_paths, auth_paths, mask_dir):
    mean_range = [round(x, 2) for x in np.arange(0.20, 0.291, 0.01)]
    area_range = [200]
    # 1. Use ALL images from both paths to maximize robustness
    val_set = [(p, "forged") for p in forg_paths] + [(p, "authentic") for p in auth_paths]
    
    print(f"🚀 Step 1: Caching probability maps for ALL {len(val_set)} images...")
    cache = []
    for p, label in tqdm(val_set):
        pil = Image.open(p).convert("RGB")
        w, h = pil.size
        
        # Get raw probability map
        prob = segment_prob_map_with_tta(pil) if USE_TTA else segment_prob_map(pil)
        
        # USE OLD MASK LOGIC: mean + 0.3*std
        mask_raw, _ = enhanced_adaptive_mask(prob) # Your function using np.mean + 0.3*np.std
        mask_resized = cv2.resize(mask_raw, (w, h), interpolation=cv2.INTER_NEAREST)
        
        # Handle Ground Truth
        if label == "forged":
            m_gt = np.load(Path(mask_dir)/f"{Path(p).stem}.npy")
            if m_gt.ndim == 3: m_gt = np.max(m_gt, axis=0)
            m_gt = (m_gt > 0).astype(np.uint8)
        else:
            m_gt = np.zeros((h, w), np.uint8) # Authentic = blank GT
            
        cache.append({"prob": prob, "mask": mask_resized, "gt": m_gt, "label": label})

    # 2. Sweep thresholds
    best_f1 = -1
    best_params = {}
    combinations = list(itertools.product(area_range, mean_range))
    
    for a_thr, m_thr in combinations:
        current_f1s = []
        for item in cache:
            mask = item["mask"]
            area = int(mask.sum()) # OLD AREA LOGIC
            
            # OLD MEAN LOGIC
            mask_small = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
            mean_in = float(item["prob"][mask_small == 1].mean()) if area > 0 else 0.0
            
            # Pipeline decision
            is_forged = (area >= a_thr and mean_in >= m_thr)
            m_pred = (mask > 0).astype(np.uint8) if is_forged else np.zeros_like(item["gt"])
            
            # F1 Calculation (Authentic silence = 1.0, noisy prediction = 0.0)
            f1 = f1_score(item["gt"].flatten(), m_pred.flatten(), 
                          zero_division=1 if item["label"] == "authentic" else 0)
            current_f1s.append(f1)
            
        avg_f1 = np.mean(current_f1s)
        if avg_f1 > best_f1:
            best_f1 = avg_f1
            best_params = {"AREA_THR": a_thr, "MEAN_THR": m_thr}
            print(f"⭐ New Best F1: {best_f1:.4f} -> AREA: {a_thr}, MEAN: {m_thr}")

    return best_params

if GRID_SEARCH:
    best_cfg = grid_search_area_mean(val_forg, val_auth, MASK_DIR)
    AREA_THR = best_cfg['AREA_THR']
    MEAN_THR = best_cfg['MEAN_THR']


from sklearn.metrics import f1_score
val_items = [(p, 1) for p in val_forg[:10]]
results = []
for p,_ in tqdm(val_items, desc="Validation forged-only"):
    pil = Image.open(p).convert("RGB")
    label, m_pred, dbg = pipeline_final(pil)
    m_gt = np.load(Path(MASK_DIR)/f"{Path(p).stem}.npy")
    if m_gt.ndim==3: m_gt=np.max(m_gt,axis=0)
    m_gt=(m_gt>0).astype(np.uint8)
    m_pred=(m_pred>0).astype(np.uint8) if m_pred is not None else np.zeros_like(m_gt)
    f1 = f1_score(m_gt.flatten(), m_pred.flatten(), zero_division=0)
    results.append((Path(p).stem, f1, dbg))
print("\n F1-score par image falsifiée:\n")
for cid,f1,dbg in results:
    print(f"{cid} — F1={f1:.4f} | area={dbg['area']} mean={dbg['mean_inside']:.3f} thr={dbg['thr']:.3f}")
print(f"\n Moyenne F1 (falsifiées) = {np.mean([r[1] for r in results]):.4f}")


In [ ]:

import os, json, cv2
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- RLE Encoder for Kaggle Submission ---
def rle_encode(mask: np.ndarray, fg_val: int = 1) -> str:
    pixels = mask.T.flatten()
    dots = np.where(pixels == fg_val)[0]
    if len(dots) == 0:
        return "authentic"
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return json.dumps([int(x) for x in run_lengths])

# --- Paths ---
TEST_DIR = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
SAMPLE_SUB = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"
OUT_PATH = "submission1.csv"

rows = []
for f in tqdm(sorted(os.listdir(TEST_DIR)), desc="Inference on Test Set"):
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)  # utilise la version améliorée

    # Sécurisation masque
    if mask is None:
        mask = np.zeros(pil.size[::-1], np.uint8)
    else:
        mask = np.array(mask, dtype=np.uint8)

    # Annotation finale
    if label == "authentic":
        annot = "authentic"
    else:
        annot = rle_encode((mask > 0).astype(np.uint8))

    rows.append({
        "case_id": Path(f).stem,
        "annotation": annot,
        "area": int(dbg.get("area", mask.sum())),
        "mean": float(dbg.get("mean_inside", 0.0)),
        "thr": float(dbg.get("thr", 0.0))
    })


sub = pd.DataFrame(rows)
ss = pd.read_csv(SAMPLE_SUB)
ss["case_id"] = ss["case_id"].astype(str)
sub["case_id"] = sub["case_id"].astype(str)
final = ss[["case_id"]].merge(sub, on="case_id", how="left")
final["annotation"] = final["annotation"].fillna("authentic")
final[["case_id", "annotation"]].to_csv(OUT_PATH, index=False)

print(f"\n✅ Saved submission file: {OUT_PATH}")
print(final.head(10))


sample_files = sorted(os.listdir(TEST_DIR))[:5]
for f in sample_files:
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)
    mask = np.array(mask, dtype=np.uint8) if mask is not None else np.zeros(pil.size[::-1], np.uint8)

    print(f"{'🔴' if label=='forged' else '🟢'} {f}: {label} | area={mask.sum()} mean={dbg.get('mean_inside', 0):.3f}")

    if label == "authentic":
        plt.figure(figsize=(5,5))
        plt.imshow(pil)
        plt.title(f"{f} — Authentic")
        plt.axis("off")
        plt.show()
    else:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(pil)
        plt.title("Original Image")
        plt.axis("off")
        plt.subplot(1,2,2)
        plt.imshow(pil)
        plt.imshow(mask, alpha=0.45, cmap="Reds")
        plt.title(f"Predicted Forged Mask\nArea={mask.sum()} | Mean={dbg.get('mean_inside', 0):.3f}")
        plt.axis("off")
        plt.show()


In [ ]:
clear()

# 第二个submission(0.325)

In [ ]:
import os, cv2, json, math, random, torch, io
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from transformers import AutoImageProcessor, AutoModel

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_DIR  = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
AUTH_DIR  = f"{BASE_DIR}/train_images/authentic"
FORG_DIR  = f"{BASE_DIR}/train_images/forged"
MASK_DIR  = f"{BASE_DIR}/train_masks"
TEST_DIR  = f"{BASE_DIR}/test_images"
DINO_PATH = "/kaggle/input/dinov2/pytorch/base/1"

IMG_SIZE = 518
BATCH_SIZE = 2
MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U52/CNNDINOv2-U52/model_seg_final.pt'

# INFERENCE UTILS
AREA_THR = 200
MEAN_THR = 0.22
USE_TTA = False
USE_ELA = True  # NEW: Enable ELA
ELA_WEIGHT = 0.3  # Weight for ELA contribution
ELA_QUALITY = 90  # JPEG quality for ELA

print(f'Device: {device}')
print(f'ELA Enabled: {USE_ELA}')

In [ ]:
def compute_ela(pil_image, quality=90):
    """Compute Error Level Analysis for an image.
    
    Like adding forensic powder to reveal hidden fingerprints,
    ELA reveals hidden manipulations by analyzing compression artifacts.
    
    Args:
        pil_image: PIL Image in RGB format
        quality: JPEG quality for re-compression (default 90)
    
    Returns:
        PIL Image showing ELA result
    """
    # Re-compress at specified quality
    buffer = io.BytesIO()
    pil_image.save(buffer, 'JPEG', quality=quality)
    buffer.seek(0)
    recompressed = Image.open(buffer).convert('RGB')
    
    # Compute difference
    original = np.array(pil_image, dtype=np.float32)
    compressed = np.array(recompressed, dtype=np.float32)
    ela = np.abs(original - compressed)
    
    # Scale for visibility (amplify differences)
    scale = 255.0 / (ela.max() + 1e-6)
    ela = np.clip(ela * scale, 0, 255).astype(np.uint8)
    
    return Image.fromarray(ela)


def compute_ela_heatmap(pil_image, quality=90):
    """Compute ELA and return as grayscale heatmap."""
    ela_img = compute_ela(pil_image, quality)
    ela_arr = np.array(ela_img, dtype=np.float32)
    # Convert to grayscale by taking max channel
    heatmap = ela_arr.max(axis=2)
    # Normalize
    heatmap = heatmap / (heatmap.max() + 1e-6)
    return heatmap


def visualize_ela(pil_image, quality=90):
    """Visualize original image and its ELA side by side."""
    ela = compute_ela(pil_image, quality)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(pil_image)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(ela)
    axes[1].set_title(f'ELA (Quality={quality})')
    axes[1].axis('off')
    
    # Heatmap version
    heatmap = compute_ela_heatmap(pil_image, quality)
    axes[2].imshow(heatmap, cmap='hot')
    axes[2].set_title('ELA Heatmap')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def detect_hidden_jpeg(pil_image):
    """Detect if a PNG/lossless image was likely originally a JPEG.
    
    Checks for 8x8 block artifacts using a simple horizontal/vertical gradient check.
    """
    arr = np.array(pil_image.convert('L'), dtype=np.float32)
    h, w = arr.shape
    if h < 16 or w < 16: return False, 0.0
    
    # Compute gradients
    grad_x = np.abs(arr[:, 1:] - arr[:, :-1])
    grad_y = np.abs(arr[1:, :] - arr[:-1, :])
    
    # Check for 8x8 periodic peaks in gradient sums (block boundaries)
    def check_periodicity(grads, axis=0):
        sums = grads.sum(axis=axis)
        if len(sums) < 16: return 0.0
        # Look at the 8-cycle variance vs local mean
        peaks = [sums[i::8].mean() for i in range(8)]
        return np.max(peaks) / (np.mean(peaks) + 1e-6)
    
    score_x = check_periodicity(grad_x, axis=0)
    score_y = check_periodicity(grad_y, axis=1)
    
    final_score = (score_x + score_y) / 2.0
    is_hjpeg = final_score > 1.2 # Empirical threshold
    
    return is_hjpeg, final_score

def visualize_ela_suitability(pil_image):
    is_hjt, score = detect_hidden_jpeg(pil_image)
    ela = compute_ela(pil_image, quality=ELA_QUALITY)
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(pil_image)
    plt.title(f"Original (H-JPEG: {is_hjt}, Score: {score:.2f})")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(ela)
    plt.title("ELA Transformation")
    plt.axis('off')
    plt.show()

In [ ]:
# MODEL (DINOv2 + Decoder)
processor = AutoImageProcessor.from_pretrained(DINO_PATH, local_files_only=True, use_fast=False)
encoder = AutoModel.from_pretrained(DINO_PATH, local_files_only=True).eval().to(device)

class DinoTinyDecoder(nn.Module):
    def __init__(self, in_ch=768, out_ch=1):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_ch, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(384, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(192, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv_out = nn.Conv2d(96, out_ch, kernel_size=1)
    
    def forward(self, f, target_size):
        x = F.interpolate(self.block1(f), size=(74, 74), mode='bilinear', align_corners=False)
        x = F.interpolate(self.block2(x), size=(148, 148), mode='bilinear', align_corners=False)
        x = F.interpolate(self.block3(x), size=(296, 296), mode='bilinear', align_corners=False)
        x = self.conv_out(x)
        x = F.interpolate(x, size=target_size, mode='bilinear', align_corners=False)
        return x
    
class DinoSegmenter(nn.Module):
    def __init__(self, encoder, processor):
        super().__init__()
        self.encoder, self.processor = encoder, processor
        for p in self.encoder.parameters(): p.requires_grad = False
        self.seg_head = DinoTinyDecoder(768,1)
        
    def forward_features(self, x):
        imgs = (x*255).clamp(0,255).byte().permute(0,2,3,1).cpu().numpy()
        inputs = self.processor(images=list(imgs), return_tensors="pt").to(x.device)
        feats = self.encoder(**inputs).last_hidden_state
        B, N, C = feats.shape
        fmap = feats[:,1:,:].permute(0,2,1)
        s = int(math.sqrt(N-1))
        fmap = fmap.reshape(B, C, s, s)
        return fmap
        
    def forward_seg(self, x):
        fmap = self.forward_features(x)
        return self.seg_head(fmap, (IMG_SIZE, IMG_SIZE))

model_seg = DinoSegmenter(encoder, processor).to(device)

if MODEL_LOC is not None and os.path.exists(MODEL_LOC):
    model_seg.load_state_dict(torch.load(MODEL_LOC, map_location=device))
    print(f"✅ Loaded pretrained model from: {MODEL_LOC}")
    model_seg.eval()

In [ ]:
@torch.no_grad()
def segment_prob_map(pil):
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    prob = torch.sigmoid(model_seg.forward_seg(x))[0,0].cpu().numpy()
    return prob

@torch.no_grad()
def segment_prob_map_with_tta(pil):
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    predictions = []
    pred_orig = torch.sigmoid(model_seg.forward_seg(x))
    predictions.append(pred_orig)
    pred_h = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[3])))
    predictions.append(torch.flip(pred_h, dims=[3]))
    pred_v = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[2])))
    predictions.append(torch.flip(pred_v, dims=[2]))
    prob = torch.stack(predictions).mean(0)[0, 0].cpu().numpy()
    return prob

def enhanced_adaptive_mask(prob, alpha_grad=0.45):
    gx = cv2.Sobel(prob, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(prob, cv2.CV_32F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)
    grad_norm = grad_mag / (grad_mag.max() + 1e-6)
    enhanced = (1 - alpha_grad) * prob + alpha_grad * grad_norm
    enhanced = cv2.GaussianBlur(enhanced, (3,3), 0)
    thr = np.mean(enhanced) + 0.3 * np.std(enhanced)
    mask = (enhanced > thr).astype(np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask, thr

def finalize_mask(prob, orig_size):
    mask, thr = enhanced_adaptive_mask(prob)
    mask = cv2.resize(mask, orig_size, interpolation=cv2.INTER_NEAREST)
    return mask, thr

In [ ]:
def pipeline_final_with_ela(pil, use_ela=True, ela_weight=0.3):
    """Enhanced pipeline with ELA integration.
    
    Like adding forensic powder to paper, ELA reveals hidden manipulations
    by highlighting regions with different compression histories.
    """
    # Get RGB prediction
    if USE_TTA:
        prob_rgb = segment_prob_map_with_tta(pil)
    else:
        prob_rgb = segment_prob_map(pil)
    
    prob_combined = prob_rgb.copy()
    ela_info = {}
    
    if use_ela:
        # Compute ELA image
        ela_img = compute_ela(pil, quality=ELA_QUALITY)
        
        # Get model prediction on ELA image
        if USE_TTA:
            prob_ela = segment_prob_map_with_tta(ela_img)
        else:
            prob_ela = segment_prob_map(ela_img)
        
        # Get direct ELA heatmap (resized to model output size)
        ela_heatmap = compute_ela_heatmap(pil, quality=ELA_QUALITY)
        ela_heatmap = cv2.resize(ela_heatmap, (IMG_SIZE, IMG_SIZE))
        
        # Combine: RGB prob + weighted ELA prediction + small ELA heatmap boost
        prob_combined = (
            (1 - ela_weight) * prob_rgb + 
            ela_weight * 0.7 * prob_ela + 
            ela_weight * 0.3 * ela_heatmap  # Direct ELA signal
        )
        
        ela_info = {
            'ela_max': float(ela_heatmap.max()),
            'ela_mean': float(ela_heatmap.mean()),
        }
    
    mask, thr = finalize_mask(prob_combined, pil.size)
    area = int(mask.sum())
    mean_inside = float(prob_combined[cv2.resize(mask,(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_NEAREST)==1].mean()) if area>0 else 0.0
    
    if area < AREA_THR or mean_inside < MEAN_THR:
        return "authentic", None, {"area": area, "mean_inside": mean_inside, "thr": thr, **ela_info}
    return "forged", mask, {"area": area, "mean_inside": mean_inside, "thr": thr, **ela_info}


def pipeline_final(pil):
    """Wrapper to use ELA-enhanced pipeline if enabled."""
    return pipeline_final_with_ela(pil, use_ela=USE_ELA, ela_weight=ELA_WEIGHT)

In [ ]:
# Test ELA visualization on forged images
if os.path.exists(FORG_DIR):
    forg_imgs = sorted([str(Path(FORG_DIR)/f) for f in os.listdir(FORG_DIR)])[:3]
    
    for img_path in forg_imgs:
        print(f"\n📸 {Path(img_path).name}")
        pil = Image.open(img_path).convert("RGB")
        visualize_ela_suitability(pil)

In [ ]:
if os.path.exists(FORG_DIR):
    test_p = sorted([str(Path(FORG_DIR)/f) for f in os.listdir(FORG_DIR)])[5]
    pil = Image.open(test_p).convert("RGB")
    
    # 1. No ELA
    l1, m1, d1 = pipeline_final_with_ela(pil, use_ela=False)
    
    # 2. With ELA
    l2, m2, d2 = pipeline_final_with_ela(pil, use_ela=True)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(pil)
    axes[0].set_title("Original")
    
    if m1 is not None: axes[1].imshow(m1)
    axes[1].set_title(f"No ELA (Area: {d1['area']})")
    
    if m2 is not None: axes[2].imshow(m2)
    axes[2].set_title(f"With ELA (Area: {d2['area']})")
    
    plt.show()

In [ ]:
def rle_encode(mask: np.ndarray, fg_val: int = 1) -> str:
    pixels = mask.T.flatten()
    dots = np.where(pixels == fg_val)[0]
    if len(dots) == 0:
        return "authentic"
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return json.dumps([int(x) for x in run_lengths])

SAMPLE_SUB = f"{BASE_DIR}/sample_submission.csv"
OUT_PATH = "submission2.csv"

rows = []
for f in tqdm(sorted(os.listdir(TEST_DIR)), desc="Inference on Test Set with ELA"):
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)
    
    if mask is None:
        mask = np.zeros(pil.size[::-1], np.uint8)
    else:
        mask = np.array(mask, dtype=np.uint8)
    
    if label == "authentic":
        annot = "authentic"
    else:
        annot = rle_encode((mask > 0).astype(np.uint8))
    
    rows.append({
        "case_id": Path(f).stem,
        "annotation": annot,
        "area": int(dbg.get("area", mask.sum())),
        "mean": float(dbg.get("mean_inside", 0.0)),
        "thr": float(dbg.get("thr", 0.0))
    })

sub = pd.DataFrame(rows)
ss = pd.read_csv(SAMPLE_SUB)
ss["case_id"] = ss["case_id"].astype(str)
sub["case_id"] = sub["case_id"].astype(str)
final = ss[["case_id"]].merge(sub, on="case_id", how="left")
final["annotation"] = final["annotation"].fillna("authentic")
final[["case_id", "annotation"]].to_csv(OUT_PATH, index=False)

print(f"\n✅ Saved submission file: {OUT_PATH}")
print(final.head(10))

In [ ]:
clear()

# 集成

In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter

def ensemble_submissions(file_score_dict, method='weighted', threshold=0.5, 
                        use_pixel_level=False, image_sizes=None):
    """
    改进版集成函数
    
    参数:
        file_score_dict: {"文件路径": 得分}
        method: 'weighted', 'majority', 'soft_weighted', 'union', 'intersection'
        threshold: 阈值
        use_pixel_level: 是否尝试像素级集成（需要额外信息）
        image_sizes: 字典，{case_id: (height, width)}，用于像素级集成
    """
    
    # 读取所有提交文件
    submissions = {}
    for file_path, score in file_score_dict.items():
        df = pd.read_csv(file_path)
        df['case_id'] = df['case_id'].astype(str)
        submissions[file_path] = df
    
    # 检查所有文件case_id是否一致
    all_case_ids = set()
    for df in submissions.values():
        all_case_ids.update(df['case_id'].tolist())
    
    # 按case_id排序，确保顺序一致
    all_case_ids = sorted(list(all_case_ids))
    
    # 方法1: 加权投票（改进版）
    if method == 'weighted':
        # 基于排名分配权重（更好地处理微小差异）
        scores = list(file_score_dict.values())
        scores_arr = np.array(scores)
        
        # 方法A: 使用排名（第一名权重最高）
        ranks = len(scores) - np.argsort(np.argsort(scores_arr))  # 1=最好
        weights = ranks / ranks.sum()
        
        # 方法B: 或者使用softmax，但调整温度参数
        # temperature = 0.1  # 更小的温度放大差异
        # weights = np.exp((scores_arr - max(scores_arr)) / temperature)
        # weights = weights / weights.sum()
        
        print("权重分配:")
        for (file, score), weight in zip(file_score_dict.items(), weights):
            print(f"  {file} (得分: {score:.4f}): {weight:.4f}")
        
        ensemble_annotations = []
        
        for case_id in all_case_ids:
            votes = []
            vote_weights = []
            forged_annotations = []  # 收集所有伪造标注
            
            for file_path, df in submissions.items():
                row = df[df['case_id'] == case_id]
                if row.empty:
                    continue
                    
                annot = row.iloc[0]['annotation']
                weight = weights[list(file_score_dict.keys()).index(file_path)]
                
                if annot == 'authentic':
                    votes.append(0)
                    vote_weights.append(weight)
                else:
                    votes.append(1)
                    vote_weights.append(weight)
                    forged_annotations.append((annot, weight))
            
            if not votes:  # 没有该case_id的数据
                ensemble_annotations.append('authentic')
                continue
                
            # 计算加权投票结果
            weighted_vote = np.average(votes, weights=vote_weights)
            
            if weighted_vote >= threshold and forged_annotations:
                # 改进：选择最优的伪造标注
                # 方案1: 选择权重最高的
                best_annot = max(forged_annotations, key=lambda x: x[1])[0]
                # 方案2: 选择面积适中的（避免极端）
                # best_annot = select_best_by_area(forged_annotations)
                ensemble_annotations.append(best_annot)
            else:
                ensemble_annotations.append('authentic')
    
    # 方法2: 改进的多数投票
    elif method == 'majority':
        ensemble_annotations = []
        
        for case_id in all_case_ids:
            annotations = []
            for df in submissions.values():
                row = df[df['case_id'] == case_id]
                if not row.empty:
                    annotations.append(row.iloc[0]['annotation'])
            
            if not annotations:
                ensemble_annotations.append('authentic')
                continue
                
            # 统计伪造数量
            forged_count = sum(1 for annot in annotations if annot != 'authentic')
            
            # 需要严格多数（>50%）
            if forged_count > len(annotations) / 2:
                # 从伪造标注中选择（可以基于附加信息）
                forged_annots = [annot for annot in annotations if annot != 'authentic']
                
                if forged_annots:
                    # 选择策略1: 第一个
                    # best_annot = forged_annots[0]
                    
                    # 选择策略2: 基于RLE面积
                    best_annot = select_by_rle_area(forged_annots, strategy='median')
                    
                    # 选择策略3: 投票给最佳模型
                    # best_model = max(file_score_dict, key=file_score_dict.get)
                    # best_annot = submissions[best_model][submissions[best_model]['case_id']==case_id].iloc[0]['annotation']
                    
                    ensemble_annotations.append(best_annot)
                else:
                    ensemble_annotations.append('authentic')
            else:
                ensemble_annotations.append('authentic')
    
    # 方法3: 并集（任何模型认为伪造就是伪造）
    elif method == 'union':
        ensemble_annotations = []
        
        for case_id in all_case_ids:
            forged_annotations = []
            for file_path, df in submissions.items():
                row = df[df['case_id'] == case_id]
                if not row.empty and row.iloc[0]['annotation'] != 'authentic':
                    forged_annotations.append(row.iloc[0]['annotation'])
            
            if forged_annotations:
                # 选择面积最大的（并集思想）
                best_annot = max(forged_annotations, key=lambda x: calculate_rle_area(x))
                ensemble_annotations.append(best_annot)
            else:
                ensemble_annotations.append('authentic')
    
    # 方法4: 交集（所有模型都认为是伪造）
    elif method == 'intersection':
        ensemble_annotations = []
        
        for case_id in all_case_ids:
            all_annotations = []
            for df in submissions.values():
                row = df[df['case_id'] == case_id]
                if not row.empty:
                    all_annotations.append(row.iloc[0]['annotation'])
            
            if not all_annotations:
                ensemble_annotations.append('authentic')
                continue
                
            # 所有模型都认为是伪造
            if all(annot != 'authentic' for annot in all_annotations):
                # 选择面积最小的（交集思想）
                best_annot = min(all_annotations, key=lambda x: calculate_rle_area(x))
                ensemble_annotations.append(best_annot)
            else:
                ensemble_annotations.append('authentic')
    
    # 创建集成后的DataFrame
    ensemble_df = pd.DataFrame({
        'case_id': all_case_ids,
        'annotation': ensemble_annotations
    })
    
    return ensemble_df


def calculate_rle_area(rle_string):
    """计算RLE编码的面积"""
    if rle_string == 'authentic':
        return 0
    try:
        runs = json.loads(rle_string)
        # RLE格式: [start1, length1, start2, length2, ...]
        area = sum(runs[i] for i in range(1, len(runs), 2))
        return area
    except:
        return 0


def select_by_rle_area(rle_list, strategy='median'):
    """根据RLE面积选择最佳标注"""
    if not rle_list:
        return 'authentic'
    
    areas = [calculate_rle_area(rle) for rle in rle_list]
    
    if strategy == 'min':
        idx = np.argmin(areas)
    elif strategy == 'max':
        idx = np.argmax(areas)
    elif strategy == 'median':
        median_area = np.median(areas)
        idx = np.argmin(np.abs(areas - median_area))
    elif strategy == 'mean':
        mean_area = np.mean(areas)
        idx = np.argmin(np.abs(areas - mean_area))
    else:
        idx = 0
    
    return rle_list[idx]


def analyze_ensemble_quality(original_subs, ensemble_df):
    """分析集成质量"""
    print("\n📊 集成分析报告")
    print("="*50)
    
    # 统计authentic数量变化
    for file_path, df in original_subs.items():
        auth_count = (df['annotation'] == 'authentic').sum()
        print(f"{file_path}: {auth_count}个authentic")
    
    ensemble_auth = (ensemble_df['annotation'] == 'authentic').sum()
    print(f"集成结果: {ensemble_auth}个authentic")
    
    # 检查一致性
    print(f"\n集成结果与各模型差异:")
    for file_path, df in original_subs.items():
        same_count = sum(ensemble_df['annotation'] == df['annotation'])
        total_count = len(ensemble_df)
        print(f"  与{file_path}一致率: {same_count/total_count:.1%}")
    
    return ensemble_auth


# 使用示例
if __name__ == "__main__":
    # 定义文件和得分
    file_scores = {
        "submission1.csv": 0.325,
        "submission2.csv": 0.325
    }
    
    # 读取原始提交文件
    original_submissions = {}
    for file_path in file_scores.keys():
        original_submissions[file_path] = pd.read_csv(file_path)
    
    # 测试不同方法
    # methods = ['weighted', 'majority', 'union', 'intersection']
    methods = ['majority']
    
    for method in methods:
        print(f"\n🔧 使用方法: {method}")
        ensemble_df = ensemble_submissions(file_scores, method=method, threshold=0.5)
        
        # 分析质量
        auth_count = analyze_ensemble_quality(original_submissions, ensemble_df)
        
        # 保存结果
        output_file = f"submission.csv"
        ensemble_df.to_csv(output_file, index=False)
        print(f"✅ 已保存: {output_file}")
        
        # 显示前几个结果
        print("前5个预测:")
        print(ensemble_df.head())
        print()